In [1]:
spark

In [26]:
import pyspark.sql

# RDD creation

In [15]:
r = range(0,10)

In [18]:
rdd1 = sc.parallelize(r)
rdd1

PythonRDD[17] at RDD at PythonRDD.scala:53

In [4]:
rdd1.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
rdd1.first()

0

In [6]:
rdd1.take(5)

[0, 1, 2, 3, 4]

In [7]:
rdd1.count()

10

In [8]:
rdd_text = sc.textFile('LICENSE_spark.txt')

In [9]:
rdd_text.take(5)

['                                 Apache License',
 '                           Version 2.0, January 2004',
 '                        http://www.apache.org/licenses/',
 '',
 '   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION']

In [57]:
'       Apache License'.strip().split(' ')

['Apache', 'License']

In [58]:
rdd_text.map(lambda x: x.strip().split(' ')).take(5)

[['Apache', 'License'],
 ['Version', '2.0,', 'January', '2004'],
 ['http://www.apache.org/licenses/'],
 [''],
 ['TERMS',
  'AND',
  'CONDITIONS',
  'FOR',
  'USE,',
  'REPRODUCTION,',
  'AND',
  'DISTRIBUTION']]

In [60]:
rdd_text.flatMap(lambda x: x.strip().split(' ')).take(25)

['Apache',
 'License',
 'Version',
 '2.0,',
 'January',
 '2004',
 'http://www.apache.org/licenses/',
 '',
 'TERMS',
 'AND',
 'CONDITIONS',
 'FOR',
 'USE,',
 'REPRODUCTION,',
 'AND',
 'DISTRIBUTION',
 '',
 '1.',
 'Definitions.',
 '',
 '"License"',
 'shall',
 'mean',
 'the',
 'terms']

In [10]:
rdd_text.count()

582

In [14]:
rdd1.glom().collect()

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24],
 [25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49],
 [50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74],
 [75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99]]

# Operations on RDD

In [16]:
rdd1.filter(lambda x: x%2 == 0)

PythonRDD[14] at RDD at PythonRDD.scala:53

In [22]:
20%10

0

In [23]:
def es_par(x):
    return x%2 == 0
rdd1.filter(es_par).collect()

[0, 2, 4, 6, 8]

In [27]:
rdd1.map(lambda x: 2*x).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [28]:
rdd1.map(lambda x: 2*x).sum()

90

In [32]:
def inflate(x):
    return [y for y in  range(1,x)]
[inflate(i) for i in[3,4]]

[[1, 2], [1, 2, 3]]

In [33]:
rdd1.map(inflate).collect()

[[],
 [],
 [1],
 [1, 2],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 4, 5, 6, 7, 8]]

In [62]:
rdd1.glom().collect()

[[0, 1], [2, 3, 4], [5, 6], [7, 8, 9]]

In [63]:
rdd1.map(inflate).glom().collect()

[[[], []],
 [[1], [1, 2], [1, 2, 3]],
 [[1, 2, 3, 4], [1, 2, 3, 4, 5]],
 [[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8]]]

In [34]:
rdd1.flatMap(inflate).collect()

[1,
 1,
 2,
 1,
 2,
 3,
 1,
 2,
 3,
 4,
 1,
 2,
 3,
 4,
 5,
 1,
 2,
 3,
 4,
 5,
 6,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8]

In [64]:
rdd1.flatMap(inflate).glom().collect()

[[],
 [1, 1, 2, 1, 2, 3],
 [1, 2, 3, 4, 1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8]]

In [36]:
a = sc.parallelize([2,2,3])
b = sc.parallelize([2,4,2])
a.intersection(b).collect()

[2]

In [52]:
c = a.map(lambda x: (x,1))
d = b.map(lambda x: (x,2))
c.union(d).collect()

[(2, 1), (2, 1), (3, 1), (2, 2), (4, 2), (2, 2)]

In [50]:
#c.union(d).groupByKey().filter(lambda k, x: (1 in x) & (2 in x) )
c.union(d).groupByKey().filter(lambda k, x: x.next() ).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 32.0 failed 1 times, most recent failure: Lost task 3.0 in stage 32.0 (TID 185, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/c/Users/jcojan/App/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/mnt/c/Users/jcojan/App/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt/c/Users/jcojan/App/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/mnt/c/Users/jcojan/App/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
TypeError: <lambda>() missing 1 required positional a

In [47]:
1 in [1,2]

True

In [61]:
rdd2 = sc.parallelize(range(8,12))
rdd1.union(rdd2).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11]

In [65]:
x = sc.parallelize([1,2,3])
y = sc.parallelize([10,11,12])
z = x.cartesian(y)
z.collect()

[(1, 10),
 (1, 11),
 (1, 12),
 (2, 10),
 (2, 11),
 (2, 12),
 (3, 10),
 (3, 11),
 (3, 12)]

## Aggregations

In [66]:
rdd1.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [67]:
rdd1.reduce(lambda x,y: x + y)

45

In [68]:
rdd1.sum()

45

In [69]:
rdd_string = sc.parallelize(['a','b','c','d'])
rdd_string.reduce(lambda x,y: x + y)

'abcd'

In [70]:
rdd1.aggregate(0, lambda x,y: x + y, lambda x,y: x + y)

45

In [74]:
rdd1.getNumPartitions()

4

In [76]:
rdd1.aggregate(0, lambda x,y: x + y, lambda x,y: x + y)

45

In [78]:
rdd1.glom().collect()

[[0, 1], [2, 3, 4], [5, 6], [7, 8, 9]]

In [77]:
rdd1.aggregate(0, lambda x,y: max(x,y), lambda x,y: x + y)

20

In [82]:
a = sc.parallelize(["negro", "azul", "blanco", "verde", "gris"], 3)

In [83]:
a.glom().collect()

[['negro'], ['azul', 'blanco'], ['verde', 'gris']]

In [84]:
a.aggregate(0, lambda acc,y: acc + len(y), lambda x,y: x + y)

24

## With keys

In [ ]:
z.keys().collect()

In [85]:
a = sc.parallelize(["negro", "azul", "blanco", "verde", "gris"], 2)
b = a.map(lambda x: (len(x),x))
b.collect()

[(5, 'negro'), (4, 'azul'), (6, 'blanco'), (5, 'verde'), (4, 'gris')]

In [92]:
b.groupByKey().collect()

[(4, <pyspark.resultiterable.ResultIterable at 0x7f8c33533f50>),
 (6, <pyspark.resultiterable.ResultIterable at 0x7f8c33533ad0>),
 (5, <pyspark.resultiterable.ResultIterable at 0x7f8c33533590>)]

In [95]:
b.groupByKey().mapValues(lambda x: [v for v in x]).collect()

[(4, ['azul', 'gris']), (6, ['blanco']), (5, ['negro', 'verde'])]

In [90]:
b.reduceByKey(lambda x,y: x + y).collect()

[(4, 'azulgris'), (6, 'blanco'), (5, 'negroverde')]

In [ ]:
a = sc.parallelize(["negro", "azul", "blanco", "verde", "gris"], 2)
b = a.groupBy(lambda x: len(x))
sorted([(x,sorted(y)) for (x,y) in b.collect()])

In [91]:
b.countByKey()

defaultdict(int, {5: 2, 4: 2, 6: 1})

In [96]:
b.mapValues(len).collect()

[(5, 5), (4, 4), (6, 6), (5, 5), (4, 4)]

In [103]:
a = sc.parallelize(["azul", "verde", "naranja", "amar"], 3)
b = a.keyBy(lambda x: len(x))
b.collect()

[(4, 'azul'), (5, 'verde'), (7, 'naranja'), (4, 'amar')]

In [98]:
c = sc.parallelize(["negro", "blanco", "gris"], 3)
d = c.keyBy(lambda x: len(x))
d.collect()

[(5, 'negro'), (6, 'blanco'), (4, 'gris')]

In [104]:
b.join(d).collect()

[(4, ('azul', 'gris')), (4, ('amar', 'gris')), (5, ('verde', 'negro'))]

In [100]:
b.leftOuterJoin(d).collect()

[(7, ('naranja', None)), (4, ('azul', 'gris')), (5, ('verde', 'negro'))]

In [101]:
b.rightOuterJoin(d).collect()

[(6, (None, 'blanco')), (4, ('azul', 'gris')), (5, ('verde', 'negro'))]

In [102]:
b.fullOuterJoin(d).collect()

[(6, (None, 'blanco')),
 (7, ('naranja', None)),
 (4, ('azul', 'gris')),
 (5, ('verde', 'negro'))]

In [105]:
a = sc.parallelize(["negro", "azul", "blaco", "verde", "gris"], 2)
b = a.map(lambda x: (len(x), x))
b.reduceByKey(lambda x,y: x + y).collect()

[(4, 'azulgris'), (5, 'negroblacoverde')]

## Words count

In [ ]:
import re
pattern = re.compile('[^[^a-zA-Z ]')

In [ ]:
rdd_text = sc.textFile('LICENSE_spark.txt')

In [ ]:
rdd_text.take(5)

**Q1:** limpiar la lista, quitar espacios y caracteres non alfabeticos

**Q2:** separar las palabras

**Q3:** cuantas palabras differentes ?

**Q4:** palabras mas frequentes ?

# Accumulator

In [ ]:
acc = sc.accumulator(0)

In [ ]:
def initiate(x):
    acc.add(1)
    return (x,1)

In [ ]:
words_pairs2 = words.map(initiate)

In [ ]:
words_pairs2.foldByKey(0, lambda x,y: x+y).take(5)

In [ ]:
acc.value

In [ ]:
words_pairs2.foreach(lambda x: acc.value)

In [ ]:
list_nt = sc.parallelize(range(1,100))

In [ ]:
list_nt.foreach(lambda x: acc.add(1))

In [ ]:
acc.value

# Broadcast

In [ ]:
brands = SparkContext.broadcast(sc,["Apache", "Spark", "Hive", "AWS"])

In [ ]:
words_pairs3 = words\
    .filter(lambda x: x in brands.value)\
    .map(initiate)

In [ ]:
words_pairs3.foldByKey(0, lambda x,y: x+y).collect()

In [ ]:
br_ints = sc.broadcast([3,5])

In [ ]:
list_nt.filter(lambda x: all([x % i == 0 for i in br_ints.value])).collect()

# Analysis on numerical RDDs

In [ ]:
import random

In [ ]:
rdd2 = sc.parallelize([random.randint(0,10) for _ in range(25)])

In [ ]:
rdd2.stats()

In [ ]:
rdd2.histogram(5)

# Partitions

In [ ]:
rdd2.getNumPartitions()

In [ ]:
def printList(l): print('['+ ','.join([str(i) for i in l]) + ']')
rdd.foreachPartition(printList) # actually printing on stout of the 

In [ ]:
#def collectList(iterator): return list(iterator)
def collectList(iterator): yield list(iterator)
#rdd.mapPartitions(lambda i: list(i)).take(3)
rdd2.mapPartitions(collectList).collect()

In [ ]:
# there is a predefined function to do that: glom()
rdd2.glom().collect()

In [ ]:
def lenIter(iterator): yield sum(1 for x in iterator)
#def lenIter(iterator): return sum(1 for x in iterator)
rdd2.mapPartitions(lenIter).collect()

In [ ]:
print(str(rdd2.coalesce(2).glom().toDebugString()).replace('\\n','\n'))
rdd2.coalesce(2).glom().collect()

In [ ]:
rdd2.coalesce(6).glom().collect()

In [ ]:
print(str(rdd2.repartition(2).glom().toDebugString()).replace('\\n','\n'))
rdd2.repartition(2).glom().collect()

In [ ]:
rdd2.repartition(6).glom().collect()

In [ ]:
rdd2.map(lambda x: (x%6, x)).partitionBy(6).glom().collect()
# idem as rdd.map(lambda x: (x,x)).partitionBy(6, partitionFunc=(lambda x: x%6)).glom().collect()

# Persistance

In [ ]:
rdd3 = sc.parallelize([random.randint(0,10) for _ in range(50)])\
    .map(lambda x: (x%5, x)).cache() #.checkpoint()

In [ ]:
grps = rdd3.groupByKey()

In [ ]:
grps.count()

In [ ]:
grps.mapValues(lambda l: [x for x in l]).collect()

In [2]:
import random

In [3]:
list = random.sample(range(1, 100), 50)
listrdd = sc.parallelize(list, 5)



In [4]:
listrdd.collect()

[72,
 62,
 88,
 46,
 84,
 93,
 76,
 97,
 43,
 81,
 40,
 8,
 95,
 29,
 86,
 83,
 75,
 19,
 28,
 10,
 45,
 9,
 4,
 61,
 37,
 89,
 57,
 23,
 53,
 15,
 17,
 34,
 68,
 31,
 52,
 30,
 55,
 36,
 13,
 11,
 20,
 2,
 77,
 21,
 99,
 90,
 58,
 1,
 50,
 25]

In [5]:
pairs = listrdd.map(lambda x: (x, x*x))
pairs.collect()

[(72, 5184),
 (62, 3844),
 (88, 7744),
 (46, 2116),
 (84, 7056),
 (93, 8649),
 (76, 5776),
 (97, 9409),
 (43, 1849),
 (81, 6561),
 (40, 1600),
 (8, 64),
 (95, 9025),
 (29, 841),
 (86, 7396),
 (83, 6889),
 (75, 5625),
 (19, 361),
 (28, 784),
 (10, 100),
 (45, 2025),
 (9, 81),
 (4, 16),
 (61, 3721),
 (37, 1369),
 (89, 7921),
 (57, 3249),
 (23, 529),
 (53, 2809),
 (15, 225),
 (17, 289),
 (34, 1156),
 (68, 4624),
 (31, 961),
 (52, 2704),
 (30, 900),
 (55, 3025),
 (36, 1296),
 (13, 169),
 (11, 121),
 (20, 400),
 (2, 4),
 (77, 5929),
 (21, 441),
 (99, 9801),
 (90, 8100),
 (58, 3364),
 (1, 1),
 (50, 2500),
 (25, 625)]

In [7]:
reduced = pairs.reduceByKey(lambda v1, v2: v1+v2, 4)
reduced.collect()

[(72, 5184),
 (88, 7744),
 (84, 7056),
 (76, 5776),
 (40, 1600),
 (8, 64),
 (28, 784),
 (4, 16),
 (68, 4624),
 (52, 2704),
 (36, 1296),
 (20, 400),
 (93, 8649),
 (97, 9409),
 (81, 6561),
 (29, 841),
 (45, 2025),
 (9, 81),
 (61, 3721),
 (37, 1369),
 (89, 7921),
 (57, 3249),
 (53, 2809),
 (17, 289),
 (13, 169),
 (77, 5929),
 (21, 441),
 (1, 1),
 (25, 625),
 (62, 3844),
 (46, 2116),
 (86, 7396),
 (10, 100),
 (34, 1156),
 (30, 900),
 (2, 4),
 (90, 8100),
 (58, 3364),
 (50, 2500),
 (43, 1849),
 (95, 9025),
 (83, 6889),
 (75, 5625),
 (19, 361),
 (23, 529),
 (15, 225),
 (31, 961),
 (55, 3025),
 (11, 121),
 (99, 9801)]

In [8]:
finalrdd = reduced.map(lambda v:"K={},V={}".format(v[0],v[1]))
finalrdd.collect()

['K=72,V=5184',
 'K=88,V=7744',
 'K=84,V=7056',
 'K=76,V=5776',
 'K=40,V=1600',
 'K=8,V=64',
 'K=28,V=784',
 'K=4,V=16',
 'K=68,V=4624',
 'K=52,V=2704',
 'K=36,V=1296',
 'K=20,V=400',
 'K=93,V=8649',
 'K=97,V=9409',
 'K=81,V=6561',
 'K=29,V=841',
 'K=45,V=2025',
 'K=9,V=81',
 'K=61,V=3721',
 'K=37,V=1369',
 'K=89,V=7921',
 'K=57,V=3249',
 'K=53,V=2809',
 'K=17,V=289',
 'K=13,V=169',
 'K=77,V=5929',
 'K=21,V=441',
 'K=1,V=1',
 'K=25,V=625',
 'K=62,V=3844',
 'K=46,V=2116',
 'K=86,V=7396',
 'K=10,V=100',
 'K=34,V=1156',
 'K=30,V=900',
 'K=2,V=4',
 'K=90,V=8100',
 'K=58,V=3364',
 'K=50,V=2500',
 'K=43,V=1849',
 'K=95,V=9025',
 'K=83,V=6889',
 'K=75,V=5625',
 'K=19,V=361',
 'K=23,V=529',
 'K=15,V=225',
 'K=31,V=961',
 'K=55,V=3025',
 'K=11,V=121',
 'K=99,V=9801']